In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
data = pd.read_csv('cleaned_data.csv') 


In [6]:
data.shape

(23315, 31)

In [7]:

# Identify numerical and categorical features
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.to_list()
numeric_features.remove('patient_nbr')  # Remove patient identifier from features
categorical_features = data.select_dtypes(include=['object']).columns.to_list()

# Define preprocessing for numerical columns (scale them)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical columns (encode them)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Define the model
rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Separate target variable and predictors
X = data.drop(['readmitted', 'patient_nbr'], axis=1)  # Exclude 'readmitted' and 'patient_nbr' from features
y = data['readmitted']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df


,precision,recall,f1-score,support
False,0.904176,0.999684,0.949534,6324.000000
True,0.333333,0.001490,0.002967,671.000000
accuracy,0.903931,0.903931,0.903931,0.903931
macro avg,0.618755,0.500587,0.476251,6995.000000
weighted avg,0.849418,0.903931,0.858734,6995.000000
